<a href="https://colab.research.google.com/github/ecocw/114_bigdata/blob/AI_demo/%E3%80%90Demo07a%E3%80%91AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_Reflection_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Reflection 的任務設計

#### 🌟 任務說明：社群媒體發文優化小幫手

**🎯 流程說明：**
1. 使用者輸入今天想分享的內容
2. `model_writer` 生成第一版社群發文（風格活潑、有趣、使用 emoji、第一人稱）
3. `model_reviewer` 檢查內容是否夠生活化、通順、有趣，並提供具體修改建議
4. `model_writer` 根據建議產出第二版
5. Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama-3.3-70b-versatile"

#【使用 Gemini】
# api_key = userdata.get('Gemini')
# model="gemini-2.5-flash"
# base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [3]:
!pip install aisuite[all]

### 2. 基本的設定

In [4]:
import aisuite as ai

In [5]:
# provider_writer = ＂Gemini＂
# model_writer="gemini-2.5-flash"

provider_reviewer = "groq"
model_reviewer = "llama-3.3-70b-versatile"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

標準回應函式

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama-3.3-70b-versatile"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「作者」和「審查員」

In [7]:
system_writer = "你是一位活潑、有趣的社群媒體幫手，擅長幫我把日常分享變得更吸睛、第一人稱風格、有情緒、有 emoji，有時還帶點小幽默。請用台灣習慣的中文回應。"
system_reviewer = "你是一位文案潤稿專家，擅長讓貼文更口語、生活化、自然有趣，請針對以下貼文給出具體修改建議。請用台灣習慣的中文回應。"

In [8]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的貼文：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更生活化、更自然。請用台灣習慣的中文, 並且只要輸出改好的文章就可以了。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [9]:
!pip install gradio

In [10]:
import gradio as gr

In [11]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 社群貼文反思幫手（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入你今天想分享的內容")
    btn = gr.Button("生成貼文 & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label="🌟 第一版貼文 (model_writer)")
        out2 = gr.Textbox(label="🧐 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label="✨ 第二版貼文 (model_writer 改寫)")

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd925ddab2d9cff328.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
